In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시계열 예측에 활용하기 위해 데이터 기본 전처리 진행. 코드 복원용

In [2]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("/content/drive/MyDrive/Zipbob_timeseries/pricedb예제data.csv", encoding='cp949')

In [5]:
data

,itemnm,kindnm,regdate,price
0,팥,붉은 팥(국산)(1kg),1996-01-03,"3,375"
1,팥,붉은 팥(국산)(1kg),1996-01-04,"3,375"
2,팥,붉은 팥(국산)(1kg),1996-01-05,"3,375"
3,팥,붉은 팥(국산)(1kg),1996-01-08,"3,375"
4,팥,붉은 팥(국산)(1kg),1996-01-09,"3,375"
...,...,...,...,...
335770,가지,가지(1kg),2011-04-21,"3,400"
335771,가지,가지(1kg),2011-04-22,"3,400"
335772,가지,가지(1kg),2011-04-25,"3,400"
335773,가지,가지(1kg),2011-04-26,"3,400"


In [6]:
data = data.drop_duplicates()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314270 entries, 0 to 331753
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   itemnm   314270 non-null  object
 1   kindnm   314270 non-null  object
 2   regdate  314270 non-null  object
 3   price    314270 non-null  object
dtypes: object(4)
memory usage: 12.0+ MB


In [8]:
data.shape # 중복행제거 21,504개

(314270, 4)

In [9]:
# 과거날짜, 최근날짜 확인
data['regdate'].min(), data['regdate'].max()

('1996-01-03', '2021-09-16')

In [10]:
# 등록된 야채 종류 itemnm기준
data['itemnm'].unique(), len(data['itemnm'].unique())

(array(['팥', '녹두', '고구마', '양배추', '시금치', '얼갈이배추', '수박', '참외', '토마토', '딸기',
        '당근', '열무', '건고추', '풋고추', '붉은고추', '양파', '파', '생강', '가지', '미나리',
        '깻잎', '부추', '피망', '파프리카', '멜론', '느타리버섯', '팽이버섯', '새송이버섯', '감귤',
        '단감', '레몬', '체리', '망고', '건멸치', '건오징어', '김', '건미역', '굴', '전복', '쌀',
        '찹쌀', '콩', '감자', '배추', '상추', '호박', '무', '피마늘', '깐마늘(국산)', '방울토마토',
        '참깨', '들깨', '땅콩', '사과', '배', '복숭아', '포도', '바나나', '참다래', '파인애플',
        '오렌지', '고등어', '갈치', '물오징어', '북어', '새우', '메밀', '오이'], dtype=object), 68)

In [11]:
# 등록된 야채 종류 kindnm기준
data['kindnm'].unique(), len(data['kindnm'].unique())

(array(['붉은 팥(국산)(1kg)', '국산(1kg)', '밤(1kg)', '양배추(1kg)', '시금치(1kg)',
        '얼갈이배추(1kg)', '수박(1개)', '참외(1kg)', '토마토(1kg)', '딸기(1kg)',
        '무세척(1kg)', '열무(1kg)', '화건(1kg)', '햇산화건(1kg)', '풋고추(1kg)',
        '붉은고추(1kg)', '양파(1kg)', '햇양파(1kg)', '대파(1kg)', '가지(1kg)',
        '미나리(1kg)', '깻잎(1kg)', '부추(1kg)', '청(1kg)', '파프리카(1kg)', '멜론(1kg)',
        '느타리버섯(1kg)', '팽이버섯(1kg)', '새송이버섯(1kg)', '감귤(1kg)', '노지(1kg)',
        '단감(1kg)', '수입(1kg)', '건멸치(1kg)', '건오징어(20마리)', '마른김(1속)',
        '건미역(1kg)', '굴(1kg)', '전복(1kg)', '일반계(1kg)', '햇일반계(1kg)',
        '흰 콩(국산)(1kg)', '수미(1kg)', '대지마(1kg)', '봄(1kg)', '고랭지(1kg)',
        '월동(1kg)', '적(1kg)', '애호박(20개)', '한지(1kg)', '깐마늘(대서)(1kg)',
        '햇깐마늘(대서)(1kg)', '깐마늘(남도)(1kg)', '햇깐마늘(남도)(1kg)', '방울토마토(1kg)',
        '백색(국산)(1kg)', '홍로(1kg)', '신고(1kg)', '백도(1kg)', '캠벨얼리(1kg)',
        '네이블 미국(1kg)', '생선(1kg)', '황태(10마리)', '흰다리(수입)(1kg)',
        '메밀(수입)(1kg)', '가을(1kg)', '가시계통(1kg)'], dtype=object), 67)

### itemnm-kindnm조합 추출하기 83개

- itemnm-kindnm조합 추출하기 83개
- item마다 kindnm이 여러개 있는 경우가 있고, item마다 동일한 단위 '국산(1kg)'처럼 중첩되므로 item으로만 분류하지 않음

In [12]:
itemlist = list(data['itemnm'].unique())

In [13]:
itemkind=[] # 리스트 생성

In [14]:
for i in itemlist :
    dataset = data[data['itemnm']==i]
    kind_unique = list(dataset['kindnm'].unique())
    for k in range(len(kind_unique)) :
        itemkind.append([i, kind_unique[k]])

In [15]:
len(itemkind)

83

In [16]:
itemkind

[['팥', '붉은 팥(국산)(1kg)'],
 ['녹두', '국산(1kg)'],
 ['고구마', '밤(1kg)'],
 ['양배추', '양배추(1kg)'],
 ['시금치', '시금치(1kg)'],
 ['얼갈이배추', '얼갈이배추(1kg)'],
 ['수박', '수박(1개)'],
 ['참외', '참외(1kg)'],
 ['토마토', '토마토(1kg)'],
 ['딸기', '딸기(1kg)'],
 ['당근', '무세척(1kg)'],
 ['열무', '열무(1kg)'],
 ['건고추', '화건(1kg)'],
 ['건고추', '햇산화건(1kg)'],
 ['풋고추', '풋고추(1kg)'],
 ['붉은고추', '붉은고추(1kg)'],
 ['양파', '양파(1kg)'],
 ['양파', '햇양파(1kg)'],
 ['파', '대파(1kg)'],
 ['생강', '국산(1kg)'],
 ['가지', '가지(1kg)'],
 ['미나리', '미나리(1kg)'],
 ['깻잎', '깻잎(1kg)'],
 ['부추', '부추(1kg)'],
 ['피망', '청(1kg)'],
 ['파프리카', '파프리카(1kg)'],
 ['멜론', '멜론(1kg)'],
 ['느타리버섯', '느타리버섯(1kg)'],
 ['팽이버섯', '팽이버섯(1kg)'],
 ['새송이버섯', '새송이버섯(1kg)'],
 ['감귤', '감귤(1kg)'],
 ['감귤', '노지(1kg)'],
 ['단감', '단감(1kg)'],
 ['레몬', '수입(1kg)'],
 ['체리', '수입(1kg)'],
 ['망고', '수입(1kg)'],
 ['건멸치', '건멸치(1kg)'],
 ['건오징어', '건오징어(20마리)'],
 ['김', '마른김(1속)'],
 ['건미역', '건미역(1kg)'],
 ['굴', '굴(1kg)'],
 ['전복', '전복(1kg)'],
 ['쌀', '일반계(1kg)'],
 ['쌀', '햇일반계(1kg)'],
 ['찹쌀', '일반계(1kg)'],
 ['콩', '흰 콩(국산)(1kg)'],
 ['감자', '수미(1kg)'],


### 결측치 처리하기

In [17]:
data1 = data.copy()

In [18]:
data1 = data1.replace('-', '0')

In [19]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314270 entries, 0 to 331753
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   itemnm   314270 non-null  object
 1   kindnm   314270 non-null  object
 2   regdate  314270 non-null  object
 3   price    314270 non-null  object
dtypes: object(4)
memory usage: 12.0+ MB


## price object타입 변환 : , 없애고 숫자로

In [20]:
data1['price'] = data1['price'].str.replace(',', '')

In [21]:
data1['price'] = data1['price'].astype('int64')

In [22]:
data1

,itemnm,kindnm,regdate,price
0,팥,붉은 팥(국산)(1kg),1996-01-03,3375
1,팥,붉은 팥(국산)(1kg),1996-01-04,3375
2,팥,붉은 팥(국산)(1kg),1996-01-05,3375
3,팥,붉은 팥(국산)(1kg),1996-01-08,3375
4,팥,붉은 팥(국산)(1kg),1996-01-09,3375
...,...,...,...,...
331749,피마늘,한지(1kg),2020-12-24,8000
331750,피마늘,한지(1kg),2020-12-28,8000
331751,피마늘,한지(1kg),2020-12-29,8000
331752,피마늘,한지(1kg),2020-12-30,8000


In [23]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314270 entries, 0 to 331753
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   itemnm   314270 non-null  object
 1   kindnm   314270 non-null  object
 2   regdate  314270 non-null  object
 3   price    314270 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 12.0+ MB


In [24]:
data1[data1['price']==0]

,itemnm,kindnm,regdate,price
62306,건고추,햇산화건(1kg),1996-01-03,0
62307,건고추,햇산화건(1kg),1996-01-04,0
62308,건고추,햇산화건(1kg),1996-01-05,0
62309,건고추,햇산화건(1kg),1996-01-08,0
62310,건고추,햇산화건(1kg),1996-01-09,0
...,...,...,...,...
326806,무,월동(1kg),2020-11-24,0
326807,무,월동(1kg),2020-11-25,0
326808,무,월동(1kg),2020-11-26,0
326809,무,월동(1kg),2020-11-27,0


## 종류별 예측을 위해 item-kind값 병합 컬럼 생성

In [25]:
data1['itemkind'] = data1['itemnm']+'-'+data1['kindnm']

In [26]:
data1

,itemnm,kindnm,regdate,price,itemkind
0,팥,붉은 팥(국산)(1kg),1996-01-03,3375,팥-붉은 팥(국산)(1kg)
1,팥,붉은 팥(국산)(1kg),1996-01-04,3375,팥-붉은 팥(국산)(1kg)
2,팥,붉은 팥(국산)(1kg),1996-01-05,3375,팥-붉은 팥(국산)(1kg)
3,팥,붉은 팥(국산)(1kg),1996-01-08,3375,팥-붉은 팥(국산)(1kg)
4,팥,붉은 팥(국산)(1kg),1996-01-09,3375,팥-붉은 팥(국산)(1kg)
...,...,...,...,...,...
331749,피마늘,한지(1kg),2020-12-24,8000,피마늘-한지(1kg)
331750,피마늘,한지(1kg),2020-12-28,8000,피마늘-한지(1kg)
331751,피마늘,한지(1kg),2020-12-29,8000,피마늘-한지(1kg)
331752,피마늘,한지(1kg),2020-12-30,8000,피마늘-한지(1kg)


## 최종 저장

In [28]:
data1.to_csv('/content/drive/MyDrive/Zipbob_timeseries/농수산DB기본전처리.csv', index=False, encoding='cp949')